In [1]:
%pip install weave

  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 5.8 MB/s eta 0:00:0000:0100:01
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached sniffio-1.3.1-py3-n

In [2]:
import weave
weave.init("weave-example")

@weave.op()
def sum_nine(value_one: int):
    return value_one + 9

@weave.op()
def multiply_two(value_two: int):
    return value_two * 2

@weave.op()
def main():
    output = sum_nine(3)
    final_output = multiply_two(output)
    return final_output

main()

Logged in as Weights & Biases user: 100386322.
View Weave data at https://wandb.ai/100386322/weave-example/weave
🍩 https://wandb.ai/100386322/weave-example/r/call/0192c59c-d50b-7451-835b-c1f0fb99adfb


24

In [4]:
import functools
import inspect
from vertexai.generative_models import GenerativeModel


# Instantiate the actual model from vertexai
model = GenerativeModel(
    "gemini-1.5-flash-002",  # Replace with your desired Gemini model ID
    system_instruction="You are an expert in transforming the content of webpages into structured markdown. You just receive the result from curl and you need to transform it into markdown.",
)

# Minimal decorator to identify and print the model dynamically
def find_model_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Execute the function
        result = func(*args, **kwargs)

        # Get the current frame
        frame = inspect.currentframe()
        try:
            # Get the locals from the frame where the decorated function was called
            locals_after = frame.f_back.f_locals

            # Find and print the model object based on attribute
            for var_name, var_value in locals_after.items():
                if callable(getattr(var_value, "generate_content", None)):
                    print(f"Detected model variable '{var_name}': {var_value}")
                    break
            else:
                print("No model with 'generate_content' method found.")
        finally:
            del frame  # Avoid reference cycles

        return result
    return wrapper

# Function that uses the model
@find_model_decorator
def main(page_content: str):
    response = model.generate_content(page_content)
    return response

# Test function
main("Sample prompt for model")

Detected model variable 'GenerativeModel': <class 'vertexai.generative_models.GenerativeModel'>


candidates {
  content {
    role: "model"
    parts {
      text: "Please provide me with the `curl` output.  I need the HTML or text content you received from the `curl` command to transform it into structured markdown.  The more context you provide, the better I can do the job.  For example, tell me:\n\n* **The URL:** Where did the data come from? This helps me understand the context and intended structure.\n* **The desired level of detail:** Do you want a simple markdown representation, or a highly structured one with headings, lists, tables, and code blocks?\n* **Specific elements to include or exclude:** Are there specific parts of the webpage you want me to focus on or ignore?\n* **Handling of images and links:** How should I handle images and links found in the webpage (e.g., should I preserve them, replace them with descriptions, or omit them entirely)?\n\nThe more information you give me, the more accurate and useful the resulting markdown will be.  A poorly formatted or inco

In [38]:
print([(a, getattr(model, a)) for a in dir(model) if not a.startswith("__")])


[('_MODEL_ROLE', 'model'), ('_USER_ROLE', 'user'), ('_cached_content', None), ('_generate_content', <bound method _GenerativeModel._generate_content of <vertexai.generative_models.GenerativeModel object at 0x11f6c9d00>>), ('_generate_content_async', <bound method _GenerativeModel._generate_content_async of <vertexai.generative_models.GenerativeModel object at 0x11f6c9d00>>), ('_generate_content_streaming', <bound method _GenerativeModel._generate_content_streaming of <vertexai.generative_models.GenerativeModel object at 0x11f6c9d00>>), ('_generate_content_streaming_async', <bound method _GenerativeModel._generate_content_streaming_async of <vertexai.generative_models.GenerativeModel object at 0x11f6c9d00>>), ('_generation_config', None), ('_llm_utility_async_client', <google.cloud.aiplatform_v1beta1.services.llm_utility_service.async_client.LlmUtilityServiceAsyncClient object at 0x11f76a3f0>), ('_llm_utility_async_client_value', <google.cloud.aiplatform_v1beta1.services.llm_utility_ser

In [31]:
import requests
from google.auth import default
from google.auth.transport.requests import Request

# Get the access token from the default application credentials
def get_access_token():
    credentials, _ = default()
    credentials.refresh(Request())
    return credentials.token

def get_sku_pricing(service_id):
    access_token = get_access_token()
    url = f"https://cloudbilling.googleapis.com/v1beta/skus/{service_id}/prices"
    
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(response.json())
        return response.json()  # Returns the JSON response
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Example service_id for Compute Engine
service_id = "D5F5-4280-60ED"
pricing_info = get_sku_pricing(service_id)

if pricing_info:
    # Print the SKU names and prices
    for sku in pricing_info.get("skus", []):
        print("SKU Name:", sku.get("description"))
        print("Pricing Info:", sku.get("pricingInfo", []))
        print("-" * 20)



{'prices': [{'name': 'skus/D5F5-4280-60ED/price', 'currencyCode': 'USD', 'valueType': 'rate', 'rate': {'tiers': [{'startAmount': {'value': '0'}, 'listPrice': {'currencyCode': 'USD', 'nanos': 7812}}], 'unitInfo': {'unit': 'count', 'unitDescription': 'count', 'unitQuantity': {'value': '1'}}, 'aggregationInfo': {'level': 'LEVEL_ACCOUNT', 'interval': 'INTERVAL_MONTHLY'}}}]}
